In [5]:
import pandas as pd
import json

df = pd.read_json("export/export_0.json")

In [6]:
def encode_traits(trait_dict_column, active_trait_df, trait_index):
    encoding_list = []
    for trait_dict in trait_dict_column:
        trait_encoding = [ 0 for i in range(len(trait_index)) ]
        for key, value in trait_dict.items():
            if key[:5] == 'Set3_':
                trait = key[5:]
            else:
                trait = key
            min_active_dict = active_trait_df.loc[active_trait_df['trait'] == trait]['min_active'].values[0]
            has_trait_flag = 0
            for k, v in min_active_dict.items():
                if value >= v[0] and value < v[1]:
                    trait_tier = int(k)
                    has_trait_flag = 1
            if has_trait_flag == 0:
                trait_tier = 0
            trait_encoding[trait_index[trait]] = trait_tier
        encoding_list.append(trait_encoding)
    return encoding_list

In [102]:
def get_active_traits(player, char_trait, trait_tier):
    active_traits = {}
    last_rounds = get_last_rounds(player)
    for rounds in last_rounds:
        stage = rounds.split('-')[0]
        board = player['round_outcomes']['by_round'][rounds]['board']
        char_seen = set()
        trait_count = {}
        for char_dict in board:
            char = char_dict['character_id']
            if char not in char_seen:
                char_seen.add(char_dict['character_id'])
                trait_list = char_trait.loc[char_trait['name'] == char]['trait'].values[0].replace('\'', '').replace(' ', '').strip('][ ').split(',')
                for trait in trait_list:
                    if trait not in trait_count:
                        trait_count[trait] = 1
                    else:
                        trait_count[trait] += 1      
        active_traits[stage] = trait_count
        
    return active_traits
    
def get_last_rounds(player):
    round_list = list(player['round_outcomes']['by_round'].keys())
    last_round_stage = []
    last_r = 0
    last_round = ''
    for rounds in round_list:
        r = int(rounds.split('-')[1])
        if last_r > r:
            last_round_stage.append(last_round)
        last_r = r
        last_round = rounds
    last_round_stage.append(round_list[-1])
    return last_round_stage


In [103]:
with open('full_mapping.json') as json_data:
    fullmap = json.load(json_data)
    
char_trait = pd.DataFrame(fullmap['character_trait_json'].items()).astype(str)
char_trait.columns = ['name', 'trait']
print(char_trait)

trait_tier = pd.DataFrame(fullmap['trait_tier_mapping_json'].items())
print(trait_tier)

                name                                          trait
0        TFT3_Thresh                       ['ManaReaver', 'Chrono']
1          TFT3_Ekko                  ['Infiltrator', 'Cybernetic']
2   TFT3_AurelionSol                          ['Starship', 'Rebel']
3          TFT3_Lulu                        ['Mystic', 'Celestial']
4         TFT3_Janna                    ['Paragon', 'StarGuardian']
5        TFT3_Soraka                     ['Mystic', 'StarGuardian']
6         TFT3_Urgot                    ['Protector', 'Battlecast']
7     TFT3_Gangplank  ['Mercenary', 'Demolitionist', 'SpacePirate']
8        TFT3_Xerath                       ['Sorcerer', 'DarkStar']
9          TFT3_Fizz                   ['Infiltrator', 'MechPilot']
10       TFT3_Irelia    ['ManaReaver', 'Blademaster', 'Cybernetic']
11       TFT3_Ezreal                          ['Blaster', 'Chrono']
12        TFT3_Riven                      ['Blademaster', 'Chrono']
13        TFT3_Fiora                  ['Blademas

In [104]:
player = {'round_outcomes': {'by_round': {'1-1': {'board': [{'character_id': 'TFT3_Fiora', 'character_name': 'Fiora', 'level': '1', 'items': [3], 'cell': 'cell_25'}], 'bench': [{'character_id': 'TFT3_Nocturne', 'character_name': 'Nocturne', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_2'}], 'opponent': None, 'type': 'PVE', 'outcome': 1.0, 'max_champion_level': 1}, '1-3': {'board': [{'character_id': 'TFT3_Fiora', 'character_name': 'Fiora', 'level': '1', 'items': [], 'cell': 'cell_25'}], 'bench': [{'character_id': 'TFT3_Nocturne', 'character_name': 'Nocturne', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_3'}], 'opponent': None, 'type': 'PVE', 'outcome': None, 'max_champion_level': 1}, '1-4': {'board': [{'character_id': 'TFT3_Fiora', 'character_name': 'Fiora', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Nocturne', 'character_name': 'Nocturne', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Illaoi', 'character_name': 'Illaoi', 'level': '1', 'items': [], 'cell': 'slot_5'}], 'opponent': None, 'type': 'PVE', 'outcome': None, 'max_champion_level': 1}, '2-1': {'board': [{'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_Fiora', 'character_name': 'Fiora', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_2'}], 'opponent': 'dekhi4712', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 1}, '2-2': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [19], 'cell': 'cell_11'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_25'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'boss cute 1 2 ', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 1}, '2-3': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_11'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_25'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'øˇÇöï kïëñˇø', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '2-4': {'board': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_11'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_25'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_2'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': '1st.T0mBoy', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '2-6': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_3'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'cell_12'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_4'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'FÇ...Hïnätä', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '2-7': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_3'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'cell_12'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_4'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': None, 'type': 'PVE', 'outcome': 1.0, 'max_champion_level': 2}, '3-1': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'cell_7'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'cell_21'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_27'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [], 'cell': 'cell_28'}], 'bench': [{'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_4'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'Omega.UStimus', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '3-2': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_3'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'cell_12'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_4'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'MôLaChầy', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '3-3': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_4'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'øˇÇöï kïëñˇø', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '3-4': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_4'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': '1st.T0mBoy', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '3-6': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_WuKong', 'character_name': 'WuKong', 'level': '1', 'items': [6], 'cell': 'cell_23'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [1], 'cell': 'slot_3'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '1', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Shaco', 'character_name': 'Shaco', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'boss cute 1 2 ', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '3-7': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_WuKong', 'character_name': 'WuKong', 'level': '1', 'items': [6], 'cell': 'cell_23'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [77], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '1', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': None, 'type': 'PVE', 'outcome': 1.0, 'max_champion_level': 2}, '4-1': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_WuKong', 'character_name': 'WuKong', 'level': '1', 'items': [], 'cell': 'cell_23'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Mordekaiser', 'character_name': 'Mordekaiser', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '1', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_8'}], 'opponent': 'Omega.UStimus', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '4-2': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [15, 19], 'cell': 'cell_23'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '1', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_MasterYi', 'character_name': 'MasterYi', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'FÇ...Hïnätä', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '4-3': {'board': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [15, 19], 'cell': 'cell_23'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '1', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Karma', 'character_name': 'Karma', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '1', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_MasterYi', 'character_name': 'MasterYi', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Jhin', 'character_name': 'Jhin', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'øˇÇöï kïëñˇø', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '4-4': {'board': [{'character_id': 'TFT3_Nautilus', 'character_name': 'Nautilus', 'level': '1', 'items': [], 'cell': 'cell_1'}, {'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '1', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [15, 19], 'cell': 'cell_23'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '2', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Jhin', 'character_name': 'Jhin', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': '1st.T0mBoy', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '4-6': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '1', 'items': [], 'cell': 'cell_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [15, 19, 44], 'cell': 'cell_23'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Jhin', 'character_name': 'Jhin', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'dekhi4712', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '4-7': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '1', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_23'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '2', 'items': [], 'cell': 'cell_27'}], 'bench': [{'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Jhin', 'character_name': 'Jhin', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': None, 'type': 'PVE', 'outcome': 1.0, 'max_champion_level': 2}, '5-1': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [], 'cell': 'cell_23'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '2', 'items': [], 'cell': 'cell_27'}], 'bench': [{'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Shen', 'character_name': 'Shen', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Jhin', 'character_name': 'Jhin', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'boss cute 1 2 ', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '5-2': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Jhin', 'character_name': 'Jhin', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'FÇ...Hïnätä', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '5-3': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Caitlyn', 'character_name': 'Caitlyn', 'level': '2', 'items': [], 'cell': 'slot_4'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_6'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_8'}, {'character_id': 'TFT3_Jhin', 'character_name': 'Jhin', 'level': '1', 'items': [], 'cell': 'slot_9'}], 'opponent': 'øˇÇöï kïëñˇø', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '5-4': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [55, 66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}], 'bench': [{'character_id': 'TFT3_Neeko', 'character_name': 'Neeko', 'level': '2', 'items': [], 'cell': 'slot_1'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '1', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_8'}], 'opponent': 'Omega.UStimus', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '5-6': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}], 'bench': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_8'}], 'opponent': 'boss cute 1 2 ', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '5-7': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [55, 66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}], 'bench': [{'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_2'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_8'}], 'opponent': None, 'type': 'PVE', 'outcome': 1.0, 'max_champion_level': 2}, '6-1': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}], 'bench': [{'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [], 'cell': 'slot_5'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_7'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '1', 'items': [], 'cell': 'slot_8'}], 'opponent': 'FÇ...Hïnätä', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}, '6-2': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3, 77], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [55, 66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}], 'bench': [{'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '1', 'items': [], 'cell': 'slot_3'}], 'opponent': 'øˇÇöï kïëñˇø', 'type': 'PVP', 'outcome': 1.0, 'max_champion_level': 2}, '6-3': {'board': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_14'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3, 77], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [55, 66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}], 'bench': [], 'opponent': 'boss cute 1 2 ', 'type': 'PVP', 'outcome': 0.0, 'max_champion_level': 2}}, 'total': {'1st.T0mBoy': {'Wins': 3, 'Encounters': 3}, 'FÇ...Hïnätä': {'Wins': 2, 'Encounters': 4}, 'MôLaChầy': {'Wins': 0, 'Encounters': 1}, 'Omega.UStimus': {'Wins': 1, 'Encounters': 3}, 'boss cute 1 2 ': {'Wins': 2, 'Encounters': 5}, 'dekhi4712': {'Wins': 2, 'Encounters': 2}, 'øˇÇöï kïëñˇø': {'Wins': 2, 'Encounters': 5}}}, 'champion_damage': {'by_round': {'1-1': {'opponent': None, 'outcome': 1.0, 'type': 'PVE', 'units_level': {'TFT3_Fiora': 1.0}, 'units_damage': {'TFT3_Fiora': 608.0}}, '1-3': {'opponent': None, 'outcome': None, 'type': 'PVE', 'units_level': {'TFT3_Fiora': 1.0, 'TFT3_JarvanIV': 1.0}, 'units_damage': {'TFT3_Fiora': 588.0, 'TFT3_JarvanIV': 375.0}}, '1-4': {'opponent': None, 'outcome': None, 'type': 'PVE', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_Fiora': 1.0, 'TFT3_JarvanIV': 1.0}, 'units_damage': {'TFT3_XinZhao': 711.0, 'TFT3_Fiora': 312.0, 'TFT3_JarvanIV': 250.0}}, '2-1': {'opponent': 'dekhi4712', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_Xayah': 1.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_XinZhao': 1.0, 'TFT3_JarvanIV': 1.0}, 'units_damage': {'TFT3_Xayah': 977.0, 'TFT3_Mordekaiser': 486.0, 'TFT3_XinZhao': 429.0, 'TFT3_JarvanIV': 253.0}}, '2-2': {'opponent': 'boss cute 1 2 ', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_Xayah': 2.0, 'TFT3_JarvanIV': 1.0}, 'units_damage': {'TFT3_XinZhao': 444.0, 'TFT3_Mordekaiser': 443.0, 'TFT3_Xayah': 2503.0, 'TFT3_JarvanIV': 249.0}}, '2-3': {'opponent': 'øˇÇöï kïëñˇø', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_JarvanIV': 1.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_Xayah': 2.0}, 'units_damage': {'TFT3_XinZhao': 717.0, 'TFT3_JarvanIV': 395.0, 'TFT3_Mordekaiser': 274.0, 'TFT3_Xayah': 2559.0}}, '2-4': {'opponent': '1st.T0mBoy', 'outcome': 1.0, 'type': 'carousel', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_JarvanIV': 1.0, 'TFT3_Karma': 1.0, 'TFT3_Xayah': 2.0, 'TFT3_Mordekaiser': 1.0}, 'units_damage': {'TFT3_XinZhao': 854.0, 'TFT3_JarvanIV': 484.0, 'TFT3_Karma': 297.0, 'TFT3_Xayah': 2607.0, 'TFT3_Mordekaiser': 230.0}}, '2-6': {'opponent': 'FÇ...Hïnätä', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_Mordekaiser': 1.0, 'TFT3_Karma': 2.0, 'TFT3_XinZhao': 1.0, 'TFT3_JarvanIV': 1.0, 'TFT3_Xayah': 2.0}, 'units_damage': {'TFT3_Mordekaiser': 660.0, 'TFT3_Karma': 544.0, 'TFT3_XinZhao': 499.0, 'TFT3_JarvanIV': 242.0, 'TFT3_Xayah': 1932.0}}, '2-7': {'opponent': None, 'outcome': 1.0, 'type': 'PVE', 'units_level': {'TFT3_Mordekaiser': 1.0, 'TFT3_Karma': 2.0, 'TFT3_JarvanIV': 1.0, 'TFT3_XinZhao': 1.0, 'TFT3_Xayah': 2.0}, 'units_damage': {'TFT3_Mordekaiser': 710.0, 'TFT3_Karma': 689.0, 'TFT3_JarvanIV': 435.0, 'TFT3_XinZhao': 320.0, 'TFT3_Xayah': 2142.0}}, '3-1': {'opponent': 'Omega.UStimus', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_JarvanIV': 1.0, 'TFT3_Karma': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_Mordekaiser': 1.0}, 'units_damage': {'TFT3_XinZhao': 600.0, 'TFT3_JarvanIV': 444.0, 'TFT3_Karma': 255.0, 'TFT3_Xayah': 1777.0, 'TFT3_Mordekaiser': 1090.0}}, '3-2': {'opponent': 'MôLaChầy', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_Xayah': 2.0, 'TFT3_JarvanIV': 1.0, 'TFT3_Caitlyn': 2.0}, 'units_damage': {'TFT3_XinZhao': 637.0, 'TFT3_Mordekaiser': 360.0, 'TFT3_Xayah': 3193.0, 'TFT3_JarvanIV': 192.0, 'TFT3_Caitlyn': 1277.0}}, '3-3': {'opponent': 'øˇÇöï kïëñˇø', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_Caitlyn': 2.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_JarvanIV': 1.0, 'TFT3_XinZhao': 1.0, 'TFT3_Xayah': 2.0}, 'units_damage': {'TFT3_Caitlyn': 906.0, 'TFT3_Mordekaiser': 575.0, 'TFT3_JarvanIV': 555.0, 'TFT3_XinZhao': 200.0, 'TFT3_Xayah': 1816.0}}, '3-4': {'opponent': '1st.T0mBoy', 'outcome': 1.0, 'type': 'carousel', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_WuKong': 1.0, 'TFT3_Caitlyn': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_JarvanIV': 2.0}, 'units_damage': {'TFT3_XinZhao': 767.0, 'TFT3_WuKong': 613.0, 'TFT3_Caitlyn': 1527.0, 'TFT3_Xayah': 1408.0, 'TFT3_Mordekaiser': 1316.0, 'TFT3_JarvanIV': 1014.0}}, '3-6': {'opponent': 'boss cute 1 2 ', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_Xayah': 2.0, 'TFT3_WuKong': 1.0, 'TFT3_XinZhao': 1.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_Caitlyn': 2.0}, 'units_damage': {'TFT3_Xayah': 754.0, 'TFT3_WuKong': 535.0, 'TFT3_XinZhao': 339.0, 'TFT3_JarvanIV': 202.0, 'TFT3_Mordekaiser': 1669.0, 'TFT3_Caitlyn': 121.0}}, '3-7': {'opponent': None, 'outcome': 1.0, 'type': 'PVE', 'units_level': {'TFT3_Xayah': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Mordekaiser': 1.0, 'TFT3_XinZhao': 1.0, 'TFT3_WuKong': 1.0, 'TFT3_Caitlyn': 2.0}, 'units_damage': {'TFT3_Xayah': 884.0, 'TFT3_JarvanIV': 791.0, 'TFT3_Mordekaiser': 539.0, 'TFT3_XinZhao': 475.0, 'TFT3_WuKong': 446.0, 'TFT3_Caitlyn': 361.0}}, '4-1': {'opponent': 'Omega.UStimus', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_JarvanIV': 2.0, 'TFT3_Caitlyn': 2.0, 'TFT3_XinZhao': 1.0, 'TFT3_Irelia': 1.0, 'TFT3_Xayah': 2.0, 'TFT3_Shen': 1.0}, 'units_damage': {'TFT3_JarvanIV': 609.0, 'TFT3_Caitlyn': 60.0, 'TFT3_XinZhao': 550.0, 'TFT3_Irelia': 4223.0, 'TFT3_Xayah': 2117.0, 'TFT3_Shen': 167.0}}, '4-2': {'opponent': 'FÇ...Hïnätä', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_XinZhao': 1.0, 'TFT3_Shen': 1.0, 'TFT3_Irelia': 1.0, 'TFT3_Caitlyn': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_JarvanIV': 2.0}, 'units_damage': {'TFT3_XinZhao': 62.0, 'TFT3_Shen': 349.0, 'TFT3_Irelia': 2365.0, 'TFT3_Caitlyn': 166.0, 'TFT3_Xayah': 1579.0, 'TFT3_JarvanIV': 102.0}}, '4-3': {'opponent': 'øˇÇöï kïëñˇø', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_Rakan': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Irelia': 1.0, 'TFT3_Caitlyn': 2.0, 'TFT3_Ashe': 1.0, 'TFT3_Shen': 1.0, 'TFT3_Xayah': 2.0}, 'units_damage': {'TFT3_Rakan': 512.0, 'TFT3_JarvanIV': 462.0, 'TFT3_Irelia': 3050.0, 'TFT3_Caitlyn': 303.0, 'TFT3_Ashe': 231.0, 'TFT3_Shen': 190.0, 'TFT3_Xayah': 1428.0}}, '4-4': {'opponent': '1st.T0mBoy', 'outcome': 1.0, 'type': 'carousel', 'units_level': {'TFT3_JarvanIV': 2.0, 'TFT3_Irelia': 1.0, 'TFT3_Rakan': 2.0, 'TFT3_Caitlyn': 2.0, 'TFT3_Ashe': 1.0, 'TFT3_Riven': 1.0, 'TFT3_Xayah': 2.0}, 'units_damage': {'TFT3_JarvanIV': 600.0, 'TFT3_Irelia': 3311.0, 'TFT3_Rakan': 1688.0, 'TFT3_Caitlyn': 1594.0, 'TFT3_Ashe': 1439.0, 'TFT3_Riven': 1403.0, 'TFT3_Xayah': 1116.0}}, '4-6': {'opponent': 'dekhi4712', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_XinZhao': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Ashe': 1.0, 'TFT3_Xayah': 2.0, 'TFT3_Neeko': 2.0, 'TFT3_Rakan': 2.0}, 'units_damage': {'TFT3_XinZhao': 76.0, 'TFT3_Irelia': 6299.0, 'TFT3_JarvanIV': 436.0, 'TFT3_Ashe': 33.0, 'TFT3_Xayah': 282.0, 'TFT3_Neeko': 1741.0, 'TFT3_Rakan': 1120.0}}, '4-7': {'opponent': None, 'outcome': 1.0, 'type': 'PVE', 'units_level': {'TFT3_Ashe': 2.0, 'TFT3_Neeko': 2.0, 'TFT3_XinZhao': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_Rakan': 2.0}, 'units_damage': {'TFT3_Ashe': 814.0, 'TFT3_Neeko': 785.0, 'TFT3_XinZhao': 567.0, 'TFT3_Irelia': 5242.0, 'TFT3_JarvanIV': 501.0, 'TFT3_Xayah': 1487.0, 'TFT3_Rakan': 1243.0}}, '5-1': {'opponent': 'boss cute 1 2 ', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_Xayah': 2.0, 'TFT3_Ashe': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_Rakan': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_XinZhao': 2.0, 'TFT3_JarvanIV': 2.0}, 'units_damage': {'TFT3_Xayah': 66.0, 'TFT3_Ashe': 654.0, 'TFT3_Irelia': 6506.0, 'TFT3_Rakan': 518.0, 'TFT3_Urgot': 2200.0, 'TFT3_XinZhao': 1325.0, 'TFT3_JarvanIV': 1086.0}}, '5-2': {'opponent': 'FÇ...Hïnätä', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_Ashe': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_Irelia': 2.0, 'TFT3_XinZhao': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_Rakan': 2.0, 'TFT3_JarvanIV': 2.0}, 'units_damage': {'TFT3_Ashe': 927.0, 'TFT3_Urgot': 8331.0, 'TFT3_Irelia': 5700.0, 'TFT3_XinZhao': 55.0, 'TFT3_Xayah': 1579.0, 'TFT3_Rakan': 1235.0, 'TFT3_JarvanIV': 102.0}}, '5-3': {'opponent': 'øˇÇöï kïëñˇø', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_Rakan': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_Ashe': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_XinZhao': 2.0}, 'units_damage': {'TFT3_Rakan': 734.0, 'TFT3_Urgot': 320.0, 'TFT3_Ashe': 123.0, 'TFT3_Irelia': 1067.0, 'TFT3_JarvanIV': 0.0, 'TFT3_Xayah': 0.0, 'TFT3_XinZhao': 0.0}}, '5-4': {'opponent': 'Omega.UStimus', 'outcome': 1.0, 'type': 'carousel', 'units_level': {'TFT3_XinZhao': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Riven': 1.0, 'TFT3_Rakan': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_Ashe': 2.0, 'TFT3_Irelia': 2.0}, 'units_damage': {'TFT3_XinZhao': 894.0, 'TFT3_JarvanIV': 622.0, 'TFT3_Riven': 527.0, 'TFT3_Rakan': 501.0, 'TFT3_Xayah': 3859.0, 'TFT3_Urgot': 2523.0, 'TFT3_Ashe': 1602.0, 'TFT3_Irelia': 1215.0}}, '5-6': {'opponent': 'boss cute 1 2 ', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_Urgot': 1.0, 'TFT3_Rakan': 2.0, 'TFT3_Xayah': 2.0, 'TFT3_XinZhao': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_Ashe': 2.0, 'TFT3_Riven': 2.0, 'TFT3_JarvanIV': 2.0}, 'units_damage': {'TFT3_Urgot': 9964.0, 'TFT3_Rakan': 812.0, 'TFT3_Xayah': 80.0, 'TFT3_XinZhao': 758.0, 'TFT3_Irelia': 4547.0, 'TFT3_Ashe': 1962.0, 'TFT3_Riven': 1611.0, 'TFT3_JarvanIV': 1308.0}}, '5-7': {'opponent': None, 'outcome': 1.0, 'type': 'PVE', 'units_level': {'TFT3_Riven': 2.0, 'TFT3_Rakan': 2.0, 'TFT3_XinZhao': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_Xayah': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Ashe': 2.0}, 'units_damage': {'TFT3_Riven': 711.0, 'TFT3_Rakan': 521.0, 'TFT3_XinZhao': 428.0, 'TFT3_Urgot': 2717.0, 'TFT3_Xayah': 2474.0, 'TFT3_Irelia': 1540.0, 'TFT3_JarvanIV': 140.0, 'TFT3_Ashe': 107.0}}, '6-1': {'opponent': 'FÇ...Hïnätä', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_XinZhao': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_Ashe': 2.0, 'TFT3_Rakan': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Riven': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_Xayah': 2.0}, 'units_damage': {'TFT3_XinZhao': 72.0, 'TFT3_Irelia': 457.0, 'TFT3_Ashe': 443.0, 'TFT3_Rakan': 42.0, 'TFT3_JarvanIV': 409.0, 'TFT3_Riven': 371.0, 'TFT3_Urgot': 2629.0, 'TFT3_Xayah': 234.0}}, '6-2': {'opponent': 'øˇÇöï kïëñˇø', 'outcome': 1.0, 'type': 'PVP', 'units_level': {'TFT3_JarvanIV': 2.0, 'TFT3_Irelia': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_Xayah': 2.0, 'TFT3_Riven': 2.0, 'TFT3_Rakan': 2.0, 'TFT3_XinZhao': 2.0, 'TFT3_Ashe': 2.0}, 'units_damage': {'TFT3_JarvanIV': 714.0, 'TFT3_Irelia': 7090.0, 'TFT3_Urgot': 4987.0, 'TFT3_Xayah': 238.0, 'TFT3_Riven': 217.0, 'TFT3_Rakan': 1847.0, 'TFT3_XinZhao': 1832.0, 'TFT3_Ashe': 1103.0}}, '6-3': {'opponent': 'boss cute 1 2 ', 'outcome': 0.0, 'type': 'PVP', 'units_level': {'TFT3_Xayah': 2.0, 'TFT3_Riven': 2.0, 'TFT3_Rakan': 2.0, 'TFT3_XinZhao': 2.0, 'TFT3_JarvanIV': 2.0, 'TFT3_Urgot': 1.0, 'TFT3_Ashe': 2.0, 'TFT3_Irelia': 2.0}, 'units_damage': {'TFT3_Xayah': 80.0, 'TFT3_Riven': 603.0, 'TFT3_Rakan': 446.0, 'TFT3_XinZhao': 360.0, 'TFT3_JarvanIV': 2770.0, 'TFT3_Urgot': 2568.0, 'TFT3_Ashe': 1967.0, 'TFT3_Irelia': 1191.0}}}, 'total': {'TFT3_Fiora': 1508, 'TFT3_JarvanIV': 15742, 'TFT3_XinZhao': 13971, 'TFT3_Xayah': 39171, 'TFT3_Mordekaiser': 8352, 'TFT3_Karma': 1785, 'TFT3_Caitlyn': 6315, 'TFT3_WuKong': 1594, 'TFT3_Irelia': 53803, 'TFT3_Shen': 706, 'TFT3_Rakan': 11219, 'TFT3_Ashe': 11405, 'TFT3_Riven': 5443, 'TFT3_Neeko': 2526, 'TFT3_Urgot': 36239}}, 'rerolls': {'by_round': {'1-3': 0, '1-4': 0, '2-1': 0, '2-2': 0, '2-3': 0, '2-4': 0, '2-6': 0, '2-7': 0, '3-1': 0, '3-2': 0, '3-3': 0, '3-4': 0, '3-6': 0, '3-7': 0, '4-1': 0, '4-2': 7, '4-3': 5, '4-4': 6, '4-6': 2, '4-7': 0, '5-1': 0, '5-2': 0, '5-3': 0, '5-4': 2, '5-6': 6, '5-7': 0, '6-1': 10, '6-2': 8, '6-3': 3}, 'total_rerolls': 49}, 'store': {'total': {'units_bought': {'unit_bought': {'TFT3_Caitlyn': 6, 'TFT3_Xayah': 5, 'TFT3_Irelia': 5, 'TFT3_Karma': 4, 'TFT3_XinZhao': 3, 'TFT3_Ashe': 3, 'TFT3_Neeko': 3, 'TFT3_JarvanIV': 3, 'TFT3_Riven': 3, 'TFT3_Rakan': 3, 'TFT3_Shaco': 2, 'TFT3_Shen': 2, 'TFT3_Jhin': 2, 'TFT3_Urgot': 1, 'TFT3_MasterYi': 1, 'TFT3_Viktor': 1, 'TFT3_Mordekaiser': 1, 'TFT3_Teemo': 1}}, 'units_shown': {'count': {'TFT3_Ahri': 5, 'TFT3_Annie': 10, 'TFT3_Ashe': 5, 'TFT3_AurelionSol': 1, 'TFT3_Bard': 9, 'TFT3_Blitzcrank': 12, 'TFT3_Caitlyn': 12, 'TFT3_Cassiopeia': 5, 'TFT3_Darius': 4, 'TFT3_Ekko': 3, 'TFT3_Ezreal': 7, 'TFT3_Fiora': 11, 'TFT3_Fizz': 5, 'TFT3_Gangplank': 2, 'TFT3_Gnar': 5, 'TFT3_Graves': 7, 'TFT3_Illaoi': 7, 'TFT3_Irelia': 7, 'TFT3_Janna': 2, 'TFT3_JarvanIV': 5, 'TFT3_Jayce': 8, 'TFT3_Jhin': 5, 'TFT3_Jinx': 8, 'TFT3_Karma': 18, 'TFT3_KogMaw': 9, 'TFT3_Leona': 2, 'TFT3_Lucian': 11, 'TFT3_Lulu': 1, 'TFT3_Malphite': 12, 'TFT3_MasterYi': 10, 'TFT3_Mordekaiser': 7, 'TFT3_Nautilus': 10, 'TFT3_Neeko': 6, 'TFT3_Nocturne': 5, 'TFT3_Poppy': 7, 'TFT3_Rakan': 6, 'TFT3_Riven': 5, 'TFT3_Rumble': 15, 'TFT3_Shaco': 8, 'TFT3_Shen': 12, 'TFT3_Soraka': 4, 'TFT3_Syndra': 7, 'TFT3_Teemo': 4, 'TFT3_Thresh': 2, 'TFT3_TwistedFate': 8, 'TFT3_Urgot': 1, 'TFT3_Vayne': 12, 'TFT3_Vi': 7, 'TFT3_Viktor': 10, 'TFT3_WuKong': 2, 'TFT3_Xayah': 6, 'TFT3_XinZhao': 8, 'TFT3_Yasuo': 10, 'TFT3_Zed': 5, 'TFT3_Ziggs': 5, 'TFT3_Zoe': 10}}}}, 'gold': {'by_round': [{'gold': '2', 'current_round': '1-3'}, {'gold': '1', 'current_round': '1-3'}, {'gold': '0', 'current_round': '1-3'}, {'gold': '2', 'current_round': '1-4'}, {'gold': '0', 'current_round': '1-4'}, {'gold': '3', 'current_round': '2-1'}, {'gold': '1', 'current_round': '2-1'}, {'gold': '2', 'current_round': '2-1'}, {'gold': '3', 'current_round': '2-1'}, {'gold': '4', 'current_round': '2-1'}, {'gold': '0', 'current_round': '2-1'}, {'gold': '1', 'current_round': '2-1'}, {'gold': '0', 'current_round': '2-1'}, {'gold': '1', 'current_round': '2-1'}, {'gold': '5', 'current_round': '2-2'}, {'gold': '4', 'current_round': '2-2'}, {'gold': '1', 'current_round': '2-2'}, {'gold': '6', 'current_round': '2-3'}, {'gold': '3', 'current_round': '2-3'}, {'gold': '0', 'current_round': '2-3'}, {'gold': '1', 'current_round': '2-3'}, {'gold': '6', 'current_round': '2-4'}, {'gold': '2', 'current_round': '2-4'}, {'gold': '3', 'current_round': '2-4'}, {'gold': '9', 'current_round': '2-6'}, {'gold': '6', 'current_round': '2-6'}, {'gold': '5', 'current_round': '2-6'}, {'gold': '4', 'current_round': '2-6'}, {'gold': '3', 'current_round': '2-6'}, {'gold': '2', 'current_round': '2-6'}, {'gold': '3', 'current_round': '2-6'}, {'gold': '9', 'current_round': '2-7'}, {'gold': '8', 'current_round': '2-7'}, {'gold': '5', 'current_round': '2-7'}, {'gold': '10', 'current_round': '2-7'}, {'gold': '11', 'current_round': '2-7'}, {'gold': '18', 'current_round': '3-1'}, {'gold': '16', 'current_round': '3-1'}, {'gold': '22', 'current_round': '3-2'}, {'gold': '20', 'current_round': '3-2'}, {'gold': '28', 'current_round': '3-3'}, {'gold': '29', 'current_round': '3-3'}, {'gold': '27', 'current_round': '3-3'}, {'gold': '28', 'current_round': '3-3'}, {'gold': '35', 'current_round': '3-4'}, {'gold': '34', 'current_round': '3-4'}, {'gold': '30', 'current_round': '3-4'}, {'gold': '26', 'current_round': '3-4'}, {'gold': '27', 'current_round': '3-4'}, {'gold': '30', 'current_round': '3-4'}, {'gold': '39', 'current_round': '3-6'}, {'gold': '36', 'current_round': '3-6'}, {'gold': '35', 'current_round': '3-6'}, {'gold': '43', 'current_round': '3-7'}, {'gold': '44', 'current_round': '3-7'}, {'gold': '43', 'current_round': '3-7'}, {'gold': '46', 'current_round': '3-7'}, {'gold': '42', 'current_round': '3-7'}, {'gold': '44', 'current_round': '3-7'}, {'gold': '42', 'current_round': '3-7'}, {'gold': '48', 'current_round': '3-7'}, {'gold': '57', 'current_round': '4-1'}, {'gold': '61', 'current_round': '4-1'}, {'gold': '63', 'current_round': '4-1'}, {'gold': '74', 'current_round': '4-2'}, {'gold': '73', 'current_round': '4-2'}, {'gold': '70', 'current_round': '4-2'}, {'gold': '68', 'current_round': '4-2'}, {'gold': '65', 'current_round': '4-2'}, {'gold': '63', 'current_round': '4-2'}, {'gold': '61', 'current_round': '4-2'}, {'gold': '58', 'current_round': '4-2'}, {'gold': '56', 'current_round': '4-2'}, {'gold': '54', 'current_round': '4-2'}, {'gold': '52', 'current_round': '4-2'}, {'gold': '50', 'current_round': '4-2'}, {'gold': '53', 'current_round': '4-2'}, {'gold': '49', 'current_round': '4-2'}, {'gold': '59', 'current_round': '4-3'}, {'gold': '57', 'current_round': '4-3'}, {'gold': '55', 'current_round': '4-3'}, {'gold': '53', 'current_round': '4-3'}, {'gold': '50', 'current_round': '4-3'}, {'gold': '46', 'current_round': '4-3'}, {'gold': '38', 'current_round': '4-3'}, {'gold': '34', 'current_round': '4-3'}, {'gold': '30', 'current_round': '4-3'}, {'gold': '26', 'current_round': '4-3'}, {'gold': '24', 'current_round': '4-3'}, {'gold': '23', 'current_round': '4-3'}, {'gold': '21', 'current_round': '4-3'}, {'gold': '19', 'current_round': '4-3'}, {'gold': '20', 'current_round': '4-3'}, {'gold': '18', 'current_round': '4-3'}, {'gold': '14', 'current_round': '4-3'}, {'gold': '10', 'current_round': '4-3'}, {'gold': '18', 'current_round': '4-4'}, {'gold': '15', 'current_round': '4-4'}, {'gold': '13', 'current_round': '4-4'}, {'gold': '21', 'current_round': '4-4'}, {'gold': '16', 'current_round': '4-4'}, {'gold': '14', 'current_round': '4-4'}, {'gold': '12', 'current_round': '4-4'}, {'gold': '10', 'current_round': '4-4'}, {'gold': '12', 'current_round': '4-4'}, {'gold': '9', 'current_round': '4-4'}, {'gold': '7', 'current_round': '4-4'}, {'gold': '5', 'current_round': '4-4'}, {'gold': '8', 'current_round': '4-4'}, {'gold': '4', 'current_round': '4-4'}, {'gold': '5', 'current_round': '4-4'}, {'gold': '10', 'current_round': '4-6'}, {'gold': '6', 'current_round': '4-6'}, {'gold': '4', 'current_round': '4-6'}, {'gold': '2', 'current_round': '4-6'}, {'gold': '0', 'current_round': '4-6'}, {'gold': '1', 'current_round': '4-6'}, {'gold': '7', 'current_round': '4-7'}, {'gold': '4', 'current_round': '4-7'}, {'gold': '5', 'current_round': '4-7'}, {'gold': '6', 'current_round': '4-7'}, {'gold': '12', 'current_round': '5-1'}, {'gold': '13', 'current_round': '5-1'}, {'gold': '20', 'current_round': '5-2'}, {'gold': '22', 'current_round': '5-2'}, {'gold': '24', 'current_round': '5-2'}, {'gold': '20', 'current_round': '5-2'}, {'gold': '21', 'current_round': '5-2'}, {'gold': '30', 'current_round': '5-3'}, {'gold': '26', 'current_round': '5-3'}, {'gold': '33', 'current_round': '5-4'}, {'gold': '36', 'current_round': '5-4'}, {'gold': '32', 'current_round': '5-4'}, {'gold': '28', 'current_round': '5-4'}, {'gold': '24', 'current_round': '5-4'}, {'gold': '20', 'current_round': '5-4'}, {'gold': '16', 'current_round': '5-4'}, {'gold': '12', 'current_round': '5-4'}, {'gold': '8', 'current_round': '5-4'}, {'gold': '4', 'current_round': '5-4'}, {'gold': '0', 'current_round': '5-4'}, {'gold': '4', 'current_round': '5-4'}, {'gold': '2', 'current_round': '5-4'}, {'gold': '0', 'current_round': '5-4'}, {'gold': '3', 'current_round': '5-4'}, {'gold': '4', 'current_round': '5-4'}, {'gold': '0', 'current_round': '5-4'}, {'gold': '5', 'current_round': '5-6'}, {'gold': '3', 'current_round': '5-6'}, {'gold': '1', 'current_round': '5-6'}, {'gold': '9', 'current_round': '5-6'}, {'gold': '7', 'current_round': '5-6'}, {'gold': '5', 'current_round': '5-6'}, {'gold': '3', 'current_round': '5-6'}, {'gold': '1', 'current_round': '5-6'}, {'gold': '2', 'current_round': '5-6'}, {'gold': '8', 'current_round': '5-7'}, {'gold': '10', 'current_round': '5-7'}, {'gold': '12', 'current_round': '5-7'}, {'gold': '13', 'current_round': '5-7'}, {'gold': '20', 'current_round': '6-1'}, {'gold': '18', 'current_round': '6-1'}, {'gold': '14', 'current_round': '6-1'}, {'gold': '10', 'current_round': '6-1'}, {'gold': '8', 'current_round': '6-1'}, {'gold': '4', 'current_round': '6-1'}, {'gold': '2', 'current_round': '6-1'}, {'gold': '0', 'current_round': '6-1'}, {'gold': '4', 'current_round': '6-1'}, {'gold': '2', 'current_round': '6-1'}, {'gold': '0', 'current_round': '6-1'}, {'gold': '4', 'current_round': '6-1'}, {'gold': '2', 'current_round': '6-1'}, {'gold': '0', 'current_round': '6-1'}, {'gold': '4', 'current_round': '6-1'}, {'gold': '2', 'current_round': '6-1'}, {'gold': '0', 'current_round': '6-1'}, {'gold': '5', 'current_round': '6-2'}, {'gold': '3', 'current_round': '6-2'}, {'gold': '1', 'current_round': '6-2'}, {'gold': '4', 'current_round': '6-2'}, {'gold': '2', 'current_round': '6-2'}, {'gold': '0', 'current_round': '6-2'}, {'gold': '4', 'current_round': '6-2'}, {'gold': '8', 'current_round': '6-2'}, {'gold': '6', 'current_round': '6-2'}, {'gold': '4', 'current_round': '6-2'}, {'gold': '2', 'current_round': '6-2'}, {'gold': '0', 'current_round': '6-2'}, {'gold': '1', 'current_round': '6-2'}, {'gold': '6', 'current_round': '6-3'}, {'gold': '4', 'current_round': '6-3'}, {'gold': '2', 'current_round': '6-3'}, {'gold': '0', 'current_round': '6-3'}]}, 'xp': {'by_round': [{'level': 1, 'current_xp': 0, 'xp_max': 2, 'current_round': '1-1', 'lobby_level': 1, 'lobby_level_median': 1, 'lobby_level_without_local': 1, 'lobby_level_median_without_local': 1}, {'level': 2, 'current_xp': 0, 'xp_max': 2, 'current_round': '1-3', 'lobby_level': 2.0, 'lobby_level_median': 2.0, 'lobby_level_without_local': 2.0, 'lobby_level_median_without_local': 2.0}, {'level': 3, 'current_xp': 0, 'xp_max': 6, 'current_round': '1-4', 'lobby_level': 3.0, 'lobby_level_median': 3.0, 'lobby_level_without_local': 3.0, 'lobby_level_median_without_local': 3.0}, {'level': 3, 'current_xp': 2, 'xp_max': 6, 'current_round': '2-1', 'lobby_level': 3.0, 'lobby_level_median': 3.0, 'lobby_level_without_local': 3.125, 'lobby_level_median_without_local': 3.0}, {'level': 3, 'current_xp': 0, 'xp_max': 10, 'current_round': '2-1', 'lobby_level': 3.0, 'lobby_level_median': 3.0, 'lobby_level_without_local': 3.125, 'lobby_level_median_without_local': 3.0}, {'level': 4, 'current_xp': 0, 'xp_max': 10, 'current_round': '2-1', 'lobby_level': 3.0, 'lobby_level_median': 3.0, 'lobby_level_without_local': 3.125, 'lobby_level_median_without_local': 3.0}, {'level': 4, 'current_xp': 2, 'xp_max': 10, 'current_round': '2-2', 'lobby_level': 3.0, 'lobby_level_median': 3.0, 'lobby_level_without_local': 3.125, 'lobby_level_median_without_local': 3.0}, {'level': 4, 'current_xp': 4, 'xp_max': 10, 'current_round': '2-3', 'lobby_level': 4.0, 'lobby_level_median': 4.0, 'lobby_level_without_local': 4.0, 'lobby_level_median_without_local': 4.0}, {'level': 4, 'current_xp': 6, 'xp_max': 10, 'current_round': '2-4', 'lobby_level': 4.0, 'lobby_level_median': 4.0, 'lobby_level_without_local': 4.125, 'lobby_level_median_without_local': 4.0}, {'level': 4, 'current_xp': 0, 'xp_max': 20, 'current_round': '2-4', 'lobby_level': 4.0, 'lobby_level_median': 4.0, 'lobby_level_without_local': 4.125, 'lobby_level_median_without_local': 4.0}, {'level': 5, 'current_xp': 0, 'xp_max': 20, 'current_round': '2-4', 'lobby_level': 4.0, 'lobby_level_median': 4.0, 'lobby_level_without_local': 4.125, 'lobby_level_median_without_local': 4.0}, {'level': 5, 'current_xp': 2, 'xp_max': 20, 'current_round': '2-6', 'lobby_level': 4.0, 'lobby_level_median': 4.0, 'lobby_level_without_local': 4.125, 'lobby_level_median_without_local': 4.0}, {'level': 5, 'current_xp': 4, 'xp_max': 20, 'current_round': '2-7', 'lobby_level': 4.357142857142857, 'lobby_level_median': 4.0, 'lobby_level_without_local': 4.4375, 'lobby_level_median_without_local': 4.0}, {'level': 5, 'current_xp': 6, 'xp_max': 20, 'current_round': '3-1', 'lobby_level': 4.892857142857143, 'lobby_level_median': 5.0, 'lobby_level_without_local': 4.90625, 'lobby_level_median_without_local': 5.0}, {'level': 5, 'current_xp': 8, 'xp_max': 20, 'current_round': '3-2', 'lobby_level': 5.142857142857143, 'lobby_level_median': 5.0, 'lobby_level_without_local': 5.125, 'lobby_level_median_without_local': 5.0}, {'level': 5, 'current_xp': 10, 'xp_max': 20, 'current_round': '3-3', 'lobby_level': 5.142857142857143, 'lobby_level_median': 5.0, 'lobby_level_without_local': 5.125, 'lobby_level_median_without_local': 5.0}, {'level': 5, 'current_xp': 12, 'xp_max': 20, 'current_round': '3-4', 'lobby_level': 5.380952380952381, 'lobby_level_median': 5.0, 'lobby_level_without_local': 5.416666666666667, 'lobby_level_median_without_local': 5.0}, {'level': 5, 'current_xp': 16, 'xp_max': 20, 'current_round': '3-4', 'lobby_level': 5.380952380952381, 'lobby_level_median': 5.0, 'lobby_level_without_local': 5.416666666666667, 'lobby_level_median_without_local': 5.0}, {'level': 5, 'current_xp': 0, 'xp_max': 32, 'current_round': '3-4', 'lobby_level': 5.380952380952381, 'lobby_level_median': 5.0, 'lobby_level_without_local': 5.416666666666667, 'lobby_level_median_without_local': 5.0}, {'level': 6, 'current_xp': 0, 'xp_max': 32, 'current_round': '3-4', 'lobby_level': 5.380952380952381, 'lobby_level_median': 5.0, 'lobby_level_without_local': 5.416666666666667, 'lobby_level_median_without_local': 5.0}, {'level': 6, 'current_xp': 2, 'xp_max': 32, 'current_round': '3-6', 'lobby_level': 5.666666666666667, 'lobby_level_median': 6.0, 'lobby_level_without_local': 5.708333333333333, 'lobby_level_median_without_local': 6.0}, {'level': 6, 'current_xp': 4, 'xp_max': 32, 'current_round': '3-7', 'lobby_level': 5.714285714285714, 'lobby_level_median': 6.0, 'lobby_level_without_local': 5.75, 'lobby_level_median_without_local': 6.0}, {'level': 6, 'current_xp': 6, 'xp_max': 32, 'current_round': '4-1', 'lobby_level': 6.035714285714286, 'lobby_level_median': 6.0, 'lobby_level_without_local': 6.03125, 'lobby_level_median_without_local': 6.0}, {'level': 6, 'current_xp': 8, 'xp_max': 32, 'current_round': '4-2', 'lobby_level': 6.380952380952381, 'lobby_level_median': 6.0, 'lobby_level_without_local': 6.333333333333333, 'lobby_level_median_without_local': 6.0}, {'level': 6, 'current_xp': 10, 'xp_max': 32, 'current_round': '4-3', 'lobby_level': 6.571428571428571, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.5625, 'lobby_level_median_without_local': 7.0}, {'level': 6, 'current_xp': 14, 'xp_max': 32, 'current_round': '4-3', 'lobby_level': 6.571428571428571, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.5625, 'lobby_level_median_without_local': 7.0}, {'level': 6, 'current_xp': 22, 'xp_max': 32, 'current_round': '4-3', 'lobby_level': 6.571428571428571, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.5625, 'lobby_level_median_without_local': 7.0}, {'level': 6, 'current_xp': 26, 'xp_max': 32, 'current_round': '4-3', 'lobby_level': 6.571428571428571, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.5625, 'lobby_level_median_without_local': 7.0}, {'level': 6, 'current_xp': 30, 'xp_max': 32, 'current_round': '4-3', 'lobby_level': 6.571428571428571, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.5625, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 2, 'xp_max': 50, 'current_round': '4-3', 'lobby_level': 6.571428571428571, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.5625, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 4, 'xp_max': 50, 'current_round': '4-4', 'lobby_level': 6.809523809523809, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.833333333333333, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 6, 'xp_max': 50, 'current_round': '4-6', 'lobby_level': 7.0, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.0, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 8, 'xp_max': 50, 'current_round': '4-7', 'lobby_level': 7.0, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.0, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 10, 'xp_max': 50, 'current_round': '5-1', 'lobby_level': 6.923076923076923, 'lobby_level_median': 7.0, 'lobby_level_without_local': 6.933333333333334, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 12, 'xp_max': 50, 'current_round': '5-2', 'lobby_level': 7.0, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.0, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 14, 'xp_max': 50, 'current_round': '5-3', 'lobby_level': 7.166666666666667, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.142857142857143, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 16, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 20, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 24, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 28, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 32, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 36, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 40, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 44, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 48, 'xp_max': 50, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 7, 'current_xp': 2, 'xp_max': 66, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 8, 'current_xp': 2, 'xp_max': 66, 'current_round': '5-4', 'lobby_level': 7.363636363636363, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.384615384615385, 'lobby_level_median_without_local': 7.0}, {'level': 8, 'current_xp': 4, 'xp_max': 66, 'current_round': '5-6', 'lobby_level': 7.333333333333333, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.454545454545454, 'lobby_level_median_without_local': 7.0}, {'level': 8, 'current_xp': 6, 'xp_max': 66, 'current_round': '5-7', 'lobby_level': 7.333333333333333, 'lobby_level_median': 7.0, 'lobby_level_without_local': 7.454545454545454, 'lobby_level_median_without_local': 7.0}, {'level': 8, 'current_xp': 8, 'xp_max': 66, 'current_round': '6-1', 'lobby_level': 7.571428571428571, 'lobby_level_median': 8.0, 'lobby_level_without_local': 7.666666666666667, 'lobby_level_median_without_local': 8.0}, {'level': 8, 'current_xp': 10, 'xp_max': 66, 'current_round': '6-2', 'lobby_level': 7.666666666666667, 'lobby_level_median': 8.0, 'lobby_level_without_local': 7.75, 'lobby_level_median_without_local': 8.0}, {'level': 8, 'current_xp': 12, 'xp_max': 66, 'current_round': '6-3', 'lobby_level': 7.666666666666667, 'lobby_level_median': 8.0, 'lobby_level_without_local': 7.666666666666667, 'lobby_level_median_without_local': 8.0}]}, 'health': {'by_round': [{'health': '100', 'current_round': '1-0'}, {'health': '99', 'current_round': '2-2'}, {'health': '98', 'current_round': '2-3'}, {'health': '87', 'current_round': '3-1'}, {'health': '80', 'current_round': '3-2'}, {'health': '68', 'current_round': '3-6'}, {'health': '62', 'current_round': '4-1'}, {'health': '52', 'current_round': '4-2'}, {'health': '44', 'current_round': '4-3'}, {'health': '27', 'current_round': '5-3'}, {'health': '6', 'current_round': '6-1'}, {'health': '0', 'current_round': '6-3'}]}, 'final': {'comp': [{'character_id': 'TFT3_Ashe', 'character_name': 'Ashe', 'level': '2', 'items': [], 'cell': 'cell_2'}, {'character_id': 'TFT3_Xayah', 'character_name': 'Xayah', 'level': '2', 'items': [19, 23], 'cell': 'cell_4'}, {'character_id': 'TFT3_Urgot', 'character_name': 'Urgot', 'level': '1', 'items': [3, 77], 'cell': 'cell_15'}, {'character_id': 'TFT3_XinZhao', 'character_name': 'XinZhao', 'level': '2', 'items': [], 'cell': 'cell_24'}, {'character_id': 'TFT3_JarvanIV', 'character_name': 'JarvanIV', 'level': '2', 'items': [55, 66, 77], 'cell': 'cell_25'}, {'character_id': 'TFT3_Rakan', 'character_name': 'Rakan', 'level': '2', 'items': [], 'cell': 'cell_26'}, {'character_id': 'TFT3_Riven', 'character_name': 'Riven', 'level': '2', 'items': [], 'cell': 'cell_27'}, {'character_id': 'TFT3_Irelia', 'character_name': 'Irelia', 'level': '2', 'items': [15, 19, 44], 'cell': 'cell_28'}], 'traits': {'Set3_Celestial': 4, 'Set3_Blademaster': 3, 'Protector': 4}, 'comp_traits': [{'name': 'Set3_Celestial', 'tier_current': 1, 'tier_total': 3, 'num_units': 4, 'style': 1}, {'name': 'Set3_Blademaster', 'tier_current': 1, 'tier_total': 3, 'num_units': 3, 'style': 1}, {'name': 'Protector', 'tier_current': 1, 'tier_total': 3, 'num_units': 4, 'style': 1}], 'bench': [], 'last_round': '6-3', 'gold_left': '0', 'gold_gained': 298, 'gold_spent': 298, 'level': 8, 'xp_progress': '12/66', 'time_survived': 2125.751, 'placement': {'rank': '4'}, 'name': 'yeukien'}}
print(get_active_traits(player, char_trait, trait_tier))

{'1': {'Blademaster': 1, 'Cybernetic': 1, 'Protector': 1, 'DarkStar': 1}, '2': {'Blademaster': 1, 'Celestial': 2, 'Mystic': 1, 'DarkStar': 3, 'Protector': 2, 'Vanguard': 1}, '3': {'Blademaster': 1, 'Celestial': 2, 'Sniper': 1, 'Chrono': 2, 'Vanguard': 2, 'Protector': 2, 'DarkStar': 2}, '4': {'Sniper': 1, 'Celestial': 4, 'Blademaster': 2, 'ManaReaver': 1, 'Cybernetic': 1, 'Protector': 4, 'DarkStar': 1, 'StarGuardian': 1}, '5': {'Sniper': 1, 'Celestial': 4, 'Blademaster': 3, 'ManaReaver': 1, 'Cybernetic': 1, 'Protector': 4, 'Battlecast': 1, 'DarkStar': 1, 'Chrono': 1}, '6': {'Sniper': 1, 'Celestial': 4, 'Blademaster': 3, 'ManaReaver': 1, 'Cybernetic': 1, 'Protector': 4, 'Battlecast': 1, 'DarkStar': 1, 'Chrono': 1}}


In [109]:
trait_count_per_stage = []
for row in df['player']:
    trait_count_per_stage.append(get_active_traits(player, char_trait, trait_tier))

print(trait_count_per_stage)

[{'1': {'Blademaster': 1, 'Cybernetic': 1, 'Protector': 1, 'DarkStar': 1}, '2': {'Blademaster': 1, 'Celestial': 2, 'Mystic': 1, 'DarkStar': 3, 'Protector': 2, 'Vanguard': 1}, '3': {'Blademaster': 1, 'Celestial': 2, 'Sniper': 1, 'Chrono': 2, 'Vanguard': 2, 'Protector': 2, 'DarkStar': 2}, '4': {'Sniper': 1, 'Celestial': 4, 'Blademaster': 2, 'ManaReaver': 1, 'Cybernetic': 1, 'Protector': 4, 'DarkStar': 1, 'StarGuardian': 1}, '5': {'Sniper': 1, 'Celestial': 4, 'Blademaster': 3, 'ManaReaver': 1, 'Cybernetic': 1, 'Protector': 4, 'Battlecast': 1, 'DarkStar': 1, 'Chrono': 1}, '6': {'Sniper': 1, 'Celestial': 4, 'Blademaster': 3, 'ManaReaver': 1, 'Cybernetic': 1, 'Protector': 4, 'Battlecast': 1, 'DarkStar': 1, 'Chrono': 1}}, {'1': {'Blademaster': 1, 'Cybernetic': 1, 'Protector': 1, 'DarkStar': 1}, '2': {'Blademaster': 1, 'Celestial': 2, 'Mystic': 1, 'DarkStar': 3, 'Protector': 2, 'Vanguard': 1}, '3': {'Blademaster': 1, 'Celestial': 2, 'Sniper': 1, 'Chrono': 2, 'Vanguard': 2, 'Protector': 2, 'Dar